In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2 #feature selection
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('/content/train (1).csv')

Missing Values in age,Embarked imputer

OHE Sex,


Scaling Column Transformer


Feature Selection

Train DT



In [ ]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [ ]:
trf = ColumnTransformer([
    ('imputer_age',SimpleImputer(),[2]), #mean filling and 2 is index of column
    ('imputer_embarked',SimpleImputer(strategy='most_frequent'),[6]), #6 column
],remainder='passthrough')
#index used as output is numpy so we call by index

In [ ]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6]), #1,6 sex,embarked
],remainder='passthrough')

In [ ]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) #0 to 8 transformation on all col as
]) #min max scaling is necessary for feature selection we will use next
#being applied on all columns

In [ ]:
trf4 = SelectKBest(score_func=chi2,k=8) #5 best features

In [ ]:
trf5 = DecisionTreeClassifier()

In [ ]:
pipe = Pipeline([
    ('trf1',trf),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                 random_state=42)

<h3> Pipleline vs Pipeline </h3>

make pipeline is simple

In [ ]:
pipe = make_pipeline(trf,trf2,trf3,trf4,trf5) #simple

pipe.fit_transform if no Model training is being done

In [ ]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputer_age',
                                                  SimpleImputer(), [2]),
                                                 ('imputer_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7edac7be4c20>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [ ]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_ #most frequent

array(['S'], dtype=object)

In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

<h1> Cross Validation Using Pipeline</h1>

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean()


np.float64(0.6391214419383433)

<h1> Grid Search </h1>

In [ ]:
params = {
    'decisiontreeclassifier__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('imputer_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imputer_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unk...ore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7edac7be4c20>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [ ]:
grid.best_score_

np.float64(0.6391214419383433)

In [ ]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 2}

<h2> Exporting The Pipeline </h2>

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

<h2> Code for train data </h2>

In [ ]:
pipe = pickle.load(open('/content/pipe.pkl','rb'))

In [ ]:
test_input2 = np.array([2,'male',31.0, 0,0,10.5,'S'],dtype=object).reshape(1,7)

In [ ]:
pipe.predict(test_input2)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])